# Assignment 6
#### Problem 2

Jérémie Engler and Laure Toullier

07.12.2023

Aim : analysis of a cantilever beam-column element subjected to a tip force

In [45]:
#imports

import numpy as np
import matplotlib.pyplot as plt
plt.style.use('bmh')
import math

np.set_printoptions(precision=5, suppress=True, linewidth=150)

Params = plt.rcParams
Params['figure.figsize'] = (14, 7) 

In [46]:
#Data

a = 200 #mm
b = 200 #mm
L = 2000 #mm (uniform XS)
E = 200000 #MPa (elastic material)
n = 5 #number of intergration point
nf = 10 #number of fibers in the XS
stress_fibers = np.ones(nf) #initial stress in each fiber

Step 1 : Implement and validate a displacement based fiber beam-column element

In [47]:
#computation of tangent section stiffness matrix and section resisting force

def ks_fs(nf, stress_fibers):

    yk=np.zeros((nf))    #array for each the coordinate of each centroïd of each fiber
    lk=np.zeros((nf,2))
    ks=np.zeros((2,2))   #tangent section stiffness matrix
    fs=np.zeros((2,1))   #section resisting force

    for i in range(nf):
        yk[i]=-b/2+b/(2*nf)+b/nf*((i+1)-1) #centroïd of each fiber
        lki=np.array([[1,yk[i]]])
        lk[i]=lki
        ks+=np.transpose(lki)@lki*E*(a*b/nf)
        fs+=np.transpose(lki)*stress_fibers[i]*(a*b/nf)

    return ks, fs

#note: as the XS is constant, ks and fs are constant along the cantilever

The numerical integration used to obtain a numerical estimate of the integral is Gauss-Lobato, with 5 integration points. The conditions that satisfy all error partial derivatives for 5 integration points are:

    w1=w5=1/10 and r1=1, r5=-1

    w2=w4=49/90 and r2=-sqrt(21)/7, r4=sqrt(21)/7 
    
    w3=32/45 and r3=0

In [48]:
#computation of the stiffness matrix

r=np.array([-1,-np.sqrt(21)/7, 0, np.sqrt(21)/7,1])
w=np.array([0.1,49/90,32/45,49/90,0.1])

def Ke(L,w,r,k_s,f_s):
    
    Ke=np.zeros((6,6))
    Qfv=np.zeros((6,1))

    for i in range(n) :
        xi=L/2*r[i]+L/2      #we integrate using a variable transformation from r to x to normalize the system: x=L/2*r+L/2 and dx=L/2*dr 
        
        B=np.array([[-1/L, 0, 0, 1/L, 0,0],
                [0, (12*xi/(L**3))-6/(L**2), 6*xi/(L**2)-4/L, 0, -12*xi/(L**3)+6/(L**2) , 6*xi/(L**2)-2/L]])
        
        kb=(B.T@k_s)@B
        Ke+=L/2*w[i]*kb

        qb=B.T@f_s
        Qfv+=L/2*qb*w[i]

    return B, Ke, Qfv




In [49]:
k_s,f_s=ks_fs(nf, stress_fibers)

B, K_e,Q_e=Ke(L,w,r,k_s,f_s)


In [50]:
print(f"k_s={k_s}")
print(f"f_s={f_s}")
print(f"The element resisting force vector is ")
print(Q_e)
print("The tangent element stiffness matrix of a displacement based fiber BC elemnt is:")
print(K_e)

k_s=[[8.00e+09 0.00e+00]
 [0.00e+00 2.64e+13]]
f_s=[[40000.]
 [    0.]]
The element resisting force vector is 
[[-40000.]
 [     0.]
 [     0.]
 [ 40000.]
 [     0.]
 [     0.]]
The tangent element stiffness matrix of a displacement based fiber BC elemnt is:
[[ 4.00e+06  0.00e+00  0.00e+00 -4.00e+06  0.00e+00  0.00e+00]
 [ 0.00e+00  3.96e+04  3.96e+07  0.00e+00 -3.96e+04  3.96e+07]
 [ 0.00e+00  3.96e+07  5.28e+10  0.00e+00 -3.96e+07  2.64e+10]
 [-4.00e+06  0.00e+00  0.00e+00  4.00e+06  0.00e+00  0.00e+00]
 [ 0.00e+00 -3.96e+04 -3.96e+07  0.00e+00  3.96e+04 -3.96e+07]
 [ 0.00e+00  3.96e+07  2.64e+10  0.00e+00 -3.96e+07  5.28e+10]]


In [51]:
#classical stiffness matrix (from week 7, slide 19)

def stiffness_elastic_beam (E,a,b,L):

    K=np.zeros((6,6))

    I=a*b**3/12
    A=a*b
    
    K[0,0]=E*A/L
    K[0,3]=-E*A/L
    K[3,0]=-E*A/L
    K[3,3]=E*A/L

    K[1,1]=K[4,4]=E*I*12/L**3
    K[1,2]=K[2,1]=E*I*6/L**2
    K[2,2]=K[5,5]=E*I*4/L
    K[5,4]=K[4,5]=-6*E*I/L**2

    K[1,4]=K[4,1]=-E*I*12/L**3
    K[1,5]=K[5,1]=E*I*6/L**2
    K[2,4]=K[4,2]=-E*I*6/L**2
    K[2,5]=K[5,2]=E*I*2/L

    return K

K_classic=stiffness_elastic_beam (E,a,b,L)
print(f"K_classic={K_classic}")


K_classic=[[ 4.00000e+06  0.00000e+00  0.00000e+00 -4.00000e+06  0.00000e+00  0.00000e+00]
 [ 0.00000e+00  4.00000e+04  4.00000e+07  0.00000e+00 -4.00000e+04  4.00000e+07]
 [ 0.00000e+00  4.00000e+07  5.33333e+10  0.00000e+00 -4.00000e+07  2.66667e+10]
 [-4.00000e+06  0.00000e+00  0.00000e+00  4.00000e+06  0.00000e+00  0.00000e+00]
 [ 0.00000e+00 -4.00000e+04 -4.00000e+07  0.00000e+00  4.00000e+04 -4.00000e+07]
 [ 0.00000e+00  4.00000e+07  2.66667e+10  0.00000e+00 -4.00000e+07  5.33333e+10]]


In [52]:
#computation of the error

def error(nf):
    k_s,f_s=ks_fs(nf, stress_fibers)
    B,K_e,Q=Ke(L,w,r,k_s,f_s)
    
    err=np.linalg.norm(K_e-K_classic)/np.linalg.norm(K_classic)*100
    return err

def error_min(err_accept):
    n_iter=20
    n=1
    while error(n)>err_accept and n<n_iter:
        n+=1
    return n


In [53]:
err=error(nf)
print(f"error={err}")
min_num_fibers=error_min(2)
err_min=error(min_num_fibers)
print(f"Minimum number of fibers for error less than 2%={min_num_fibers}")
print(f"For a minimum number of fiber less than 2%, the error is: {err_min}")

error=0.9999999955000101
Minimum number of fibers for error less than 2%=8
For a minimum number of fiber less than 2%, the error is: 1.5624999929687748


Step 2: Conduct a load control analysis with your beam-column element

Aim: create a cantilever model using displacement-based fiber beam-column element.

In [54]:
nf=min_num_fibers
V_end=450000
q=np.transpose(np.array([[0,0,0,0,0,0]]))
Q_ext=np.transpose(np.array([[0,0,0,0,V_end,0]]))
num_incr=30 #number of increments
tol=10**(-8)
max_iter=16
iteration_counter=0
stress_fibers=np.zeros((nf)) #new stress in each fiber for new number of fibers

In [55]:
#newton raphson method

#initialisation
k_s, f_s=ks_fs(nf, np.zeros((nf)))
B, K_e, Q_e= Ke(L,w,r,k_s,f_s)
#ds=B@q

print(f"ks0={k_s}")
print(f"B0={B}")

#def newton_raphson(Q_ext, num_incr, max_iter, K_e, B, Q_e,q,a,b,L,nf,E, stress_fibers):

force=[]
disp=[]
iteration_counter=0
stress_fibers=[]

for j in range(num_incr-1):
    print(f"%%%%%%%%%%%%%%%%%%%%%%%%%%{j}th increment%%%%%%%%%%%%%%%%%%%%%%%%%%")
       
    iteration_counter+=1

    Q_ext_i=(j+1)/num_incr*Q_ext

    for i in range(max_iter):
        print(f"%%%%%%%%%%%%%%%%%%%%%%%%%{i}th iteration%%%%%%%%%%%%%%%%%%%%%%%%%%")
          
        K_ff=K_e[3:6,3:6] #we only consider the free DOF from Ke

        R=Q_ext_i-Q_e
        R_f=R[3:6,:] #we only consider the free DOF from R

        #displacement
        delta_q_f=np.linalg.solve(K_ff,R_f) #displacement increment
        print(f"delta_q_f={delta_q_f}")

        deltaqf=np.zeros((6))
        deltaqf[3]=delta_q_f[0]
        deltaqf[4]=delta_q_f[1]
        deltaqf[5]=delta_q_f[2]

        q=q+deltaqf.reshape((6,1))
        print(f"q={q}")

        #section deformation
        ds=B@q 
        print(f"ds={ds}")
        
        #stresses in each fiber
        eps_fibers=np.zeros((nf))
        yk=np.zeros((nf))

        for i in range(nf):
            yk[i]=-b/2+b/(2*nf)+b/nf*((i+1)-1) #centroïd of each fiber
            eps_fibers[i]=ds[0]+ds[1]*yk[i]
            stress_fibers[i]=eps_fibers[i]*E   #the material is considered elastic 
                
        print(f"yk={yk}")
        print(f"strain in fibers={eps_fibers}")
        print(f"stress in fibers={stress_fibers}")

        k_s, f_s=ks_fs(nf, stress_fibers)
        B, K_e, Q_e= Ke(L,w,r,k_s,f_s)
            
        print(f"Qe={Q_e}")

        norm_R=np.linalg.norm(R)
        norm_Qext=np.linalg.norm(Q_ext_i)

        if norm_R/norm_Qext<tol or j>max_iter:
            break #if the tolerance is reached, we need to go "out of the loop" to go to the newt load iteration
        if norm_R/norm_Qext>tol and j==max_iter-1:
            #print("no convergence")
            raise ValueError("No convergence !")
        else:
            force.append(Q_e)
            disp.append(q)
            j+=1

    #return q, Q_e, force, disp
    

ks0=[[8.000e+09 0.000e+00]
 [0.000e+00 2.625e+13]]
B0=[[-0.0005  0.      0.      0.0005  0.      0.    ]
 [ 0.      0.      0.001   0.     -0.      0.002 ]]
%%%%%%%%%%%%%%%%%%%%%%%%%%0th increment%%%%%%%%%%%%%%%%%%%%%%%%%%
%%%%%%%%%%%%%%%%%%%%%%%%%0th iteration%%%%%%%%%%%%%%%%%%%%%%%%%%
delta_q_f=[[0.     ]
 [1.52381]
 [0.00114]]
q=[[0.     ]
 [0.     ]
 [0.     ]
 [0.     ]
 [1.52381]
 [0.00114]]
ds=[[ 0.]
 [-0.]]


C:\Users\ltoul\AppData\Local\Temp\ipykernel_23872\3512492340.py:38: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  deltaqf[3]=delta_q_f[0]
C:\Users\ltoul\AppData\Local\Temp\ipykernel_23872\3512492340.py:39: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  deltaqf[4]=delta_q_f[1]
C:\Users\ltoul\AppData\Local\Temp\ipykernel_23872\3512492340.py:40: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  deltaqf[5]=delta_q_f[2]
C:\Users\ltoul\AppData\Local\Temp\ipykernel_23872\35124

IndexError: list assignment index out of range

In [ ]:
print(f"force={force}")
print(f"disp={disp}")

%%%%%%%%%%%%%%%%%%%%%%%%%%0th increment%%%%%%%%%%%%%%%%%%%%%%%%%%
%%%%%%%%%%%%%%%%%%%%%%%%%0th iteration%%%%%%%%%%%%%%%%%%%%%%%%%%
delta_q_f=[[0.     ]
 [1.52381]
 [0.00114]]
q=[[0.     ]
 [0.     ]
 [0.     ]
 [0.     ]
 [1.52381]
 [0.00114]]
ds=[[ 0.]
 [-0.]]


C:\Users\ltoul\AppData\Local\Temp\ipykernel_20496\3574266281.py:38: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  deltaqf[3]=delta_q_f[0]
C:\Users\ltoul\AppData\Local\Temp\ipykernel_20496\3574266281.py:39: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  deltaqf[4]=delta_q_f[1]
C:\Users\ltoul\AppData\Local\Temp\ipykernel_20496\3574266281.py:40: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  deltaqf[5]=delta_q_f[2]
C:\Users\ltoul\AppData\Local\Temp\ipykernel_20496\35742

IndexError: list assignment index out of range

In [ ]:
#Generalisation

steps = 1
lams =  np.linspace(1,load_incr)/load_incr
P_applied = np.zeros((load_incr, DOF))


#lists to generate later the graphs !
u1 = []
u2 = []
p1 = []
p2 = []
norm_r_print = []
iteration = []

iteration_counter = 0

#loops:
for i in range (load_incr):
    print(f"################# Nouvelle itération sur {i=} ######################")
    P_applied[i,:]=P.reshape((1,DOF))*(i+1)/load_incr
    print("P_applied", P_applied[i,:])

    print(f'delta_u={du}')
    p1.append(P_applied[i,2])
    p2.append(P_applied[i,3])

    for j in range(iter_lim):
        print(f"%%%%%%%%%%%%%% Nouvelle itération sur {j=} %%%%%%%%%%%%%%%%%")
        iteration_counter += 1

        #displacement:
        u=u+du
        print(f'u={u}')

        #internal force
        u_red, u_bar, eps=strain (u, L, T, nb_members, Connectivity)
        print(f'u_red={u_red}')
        print(f'u_bar={u_bar}')
        print(f'strain={eps}')
        Pr=internal_force(Connectivity, area, alpha, E0, u, L, T, sigma_0, eps_0, nb_members)

        #residual force
        R[2:4]=P_applied[i,2:4].reshape((1,2))-Pr[2:4]
        print(f'PR={Pr}')
        print(f'R={R}')
        norm_R=np.linalg.norm(R) 
        norm_r_print.append(norm_R)
        iteration.append(iteration_counter)

        #tangent stiffness
        K_global, K_t_global = stiffness (E, E0, area, nb_members, DOF, Connectivity, T, L) #new tangent stiffness matrix (K_global remains the same)
 
        if norm_R < tol:
            print(">>>>>>>>>>>>> norm_R < tol")
            u1.append(u[2])
            u2.append(u[3])
            break #if the tolerance is reached, we need to go "out of the loop" to go to the newt load iteration

        if norm_R>tol and j==iter_lim-1:
            raise ValueError("No convergence !")
        
        else:
            #Update the strain-increment delta_u in global coordinates:
            r=R.reshape((3,2))
            du=displacement(support, act_P, r, K_t_global, DOF)
            print(f'du={du}')

NameError: name 'load_incr' is not defined